# Chapter 4 - Hyperparameter Tuning

## In this notebook we will:

  - Connect to your workspace.
  - Create a virtual environment and leverage in this notebook
  - Explore the dataset
  - Data cleansing and analysis work
  - Feature Engineering
  - Register the cleansed data
  - Explore leveraging an MLTable
  - Run a job that leverages
    - sklearn pipeline for data transformation
    - `mlflow.autolog` for capturing *training* and *test* metrics
    - log additional metrics to a given job run
  - download and use a model that is created as a result of the job
  - Use `Sweep` to search for the best hyperparamers for a model

## Setting yourself up for success

- When creating a model, one of the major obstacles is having an environment that has the required dependencies.  We will create and register an AML environment and use on our compute instance.  This will allow us to leverage the model we build on a compute cluster on our compute instance.  The same packages and versions leveraged to build the model will be used to consume the model later in this notebook

Steps to setup our environment include:
- Connecting to our workspace
- Defining and registering the environment
- Making the environment available to our compute instance 
- Making the environment available to our jupyter notebook

Let's get started

Select **Kernel** > **Change Kernel** > **Python 3.10 - SDK V2**

In [ ]:
import azure.ai.ml
print(azure.ai.ml._version.VERSION)

In [ ]:
#import required libraries
import pandas as pd
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml.entities import Environment, BuildContext

## Connecting to your workspace

Once you connect to your workspace, you will create a new cpu target which you will provide an environment to.

- We should be able to connect with the MLClient using your credentials in the config of the compute instance.  You should not keep your subscription id, resource group or workspace name visible in your code

In [ ]:
subscription_id = ''
resource_group = ''
workspace = ''

ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

## Setup enviroment

### Creating environment from docker image with a conda YAML

Azure ML allows you to leverage curated environments, as well as to build your own environment from:

    - existing docker image
    - base docker image with a conda yml file to customize
    - a docker build content
    
We will proceed with creating an environment from a docker build plus a conda yml file.

In [ ]:
import os
script_folder = os.path.join(os.getcwd(), "conda-yamls")
print(script_folder)
os.makedirs(script_folder, exist_ok=True)

### Create job environment in a yml file

- At the time of writing this, the versions of `mlflow`, `azure-ai-ml` `mltable` and `azureml-mlflow` where set accordingly below.  They have been set to ensure stability of the notebook, but please update them according to the lastest package versions.

In [ ]:
%%writefile conda-yamls/job_env.yml
name: job_env
dependencies:
- python=3.10
- scikit-learn=1.1.3
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - mlflow==2.0.1
  - azure-ai-ml==1.1.2
  - mltable==1.0.0
  - azureml-mlflow==1.48.0

### Getting the most current and up-to-date base image

Default images are always changing.  
Note the base image is defined in the property `image` below.  These images are defined at [https://hub.docker.com/_/microsoft-azureml](https://hub.docker.com/_/microsoft-azureml)

The current image we have selected for this notebook is `mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04`, but based on image availability, that will change in the future.  In additon, note the python version specified in your conda environment file is `python=3.10`, as this will evolve over time as well.  Currently `MLTable` is not available in python 3.10, but as that changes, we encourage you to update python version as well.

In [ ]:
env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    conda_file="conda-yamls/job_env.yml",
    name="job_base_env",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env_docker_conda)

### Use your virtual environment in this notebook

We can actually use that virtual environment on our compute instance and in this very jupyter notebook.
Open a terminal session, and cd into your conda-yamls folder and run the following commands:

```
cd ML-Engineering-with-Azure-Machine-Learning-Service/
cd 'chapter 4'
cd conda-yamls/
conda env create -f job_env.yml
conda activate job_env
ipython kernel install --user --name job_env --display-name "job_env"
```
* After the environment has been made available to Jupyter, Refresh this session (F5, or Hit refresh on your browser)

When you go to your `Kernel` -> `Change Kernel`, it will be available to select.  You will have to rerun the notebook, but when you download the model, you will be using all of the correct versions of libraries.

*Note to remove an environment with conda leverage 
```
conda env remove -n job_env
```

# Explore Dataset

You're going to use a Python script to train a machine learning model based on the Titanic datset found in your data folder.  

In [ ]:
df= pd.read_csv('./data/titanic.csv')
print(df.shape)
print(df.columns)

## View Data

In [ ]:
df.head(5)

### Dataset field information

- **PassengerId**: (remove) Should be removed from model as they are some sort of id.
- **Pclass**: (keep) locates folks on ship *Pclass: 1st = Upper, 2nd = Middle, 3rd = Lower*
- **Name**: (remove) maybe found useful if keeping the surname, but for basic model will remove
- **Sex**: (keep) due to lifeboat priority, will likely be useful
- **Age**: (keep)important due to lifeboat priority
- **SibSp**: (keep) maybe useful, relatives will likely help others
- **Parch**: (keep) maybe useful, relatives will likely help others
- **Ticket**: (remove)
- **Fare**: (remove covered by class)
- **Cabin**: (keep) can be useful in relation of where the cabins are positioned on the ship
- **Embarked**: (keep) useful because all listed embark happened before the disaster

# Data Engineering

## Data Cleansing

We will begin by evaluating the null values in the dataset.  Note that Age, Fare and Cabin contain null values in the dataset.

In [ ]:
df.isnull().sum()

In [ ]:
import matplotlib.pyplot as plt

columns_missing = df.isna().sum().where(lambda x : x > 0).dropna()

ax = columns_missing \
.plot(kind='bar', alpha=0.9, title='Columns Missing Values', table=True)
ax.xaxis.set_visible(False) # hide x axis labels

for x in ax.patches:
    ax.text(x.get_x()+.1, x.get_height()+5, \
            str(round((x.get_height()/df.shape[0])*100, 1))+'%')
plt.show()

## Prepare data for an experiment
In the previous notebook, the data was leveraged directly from the folder on the compute instance.  We will be submitting an experiment to a compute cluster, so we will register the dataset so it will be stored in the blob storage associated with the AML workspace

### Stategy:

- For the Age, we will replace the missing values with the medians of each group
- For cabin we will mark it as X given this is probably an important feature that we would want to include.
- For Embarked,given there are only 2 rows missing this value, we will set these to a value of S

## Cleaning Age Column

Note that Age, a column that has missing data, will likely be impacted by class, as people are more established,their age will likely increase, so to replace these values, we will group by class and sex, calculate a median value and replace the na values in the dataset with the mean

In [ ]:
display(df.groupby(['Pclass', 'Sex'])['Age'].count())

display(df.groupby(['Pclass', 'Sex'])['Age'].median())

In [ ]:
df['Age'] = df.groupby(['Pclass', 'Sex'],group_keys=False)['Age'].apply(lambda x: x.fillna(x.median()))
df.isnull().sum()

In [ ]:
print(df['Sex'].unique())
df['Sex']= df['Sex'].apply(lambda x: x[0] if pd.notnull(x) else 'X')
print(df['Sex'].unique())

In [ ]:
df['Loc']= df['Cabin'].apply(lambda x: x[0] if pd.notnull(x) else 'X')
df[['Loc', 'Survived']].groupby('Loc')['Survived'].mean().plot(kind= 'bar')
plt.show()


In [ ]:
df.drop(['Cabin', 'Ticket'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
df['Embarked'] = df['Embarked'].fillna('S')

# Feature Engineering
## Create a Group Size

In [ ]:
df.loc[:,'GroupSize'] = 1 + df['SibSp'] + df['Parch']

## Fill Missing Embarded with value of S

In [ ]:
df.isnull().sum()

In [ ]:
LABEL = 'Survived'
columns_to_keep = ['Pclass', 'Sex','Age', 'Fare', 'Embared', 'Deck', 'GroupSize']
columns_to_drop = ['Name','SibSp', 'Parch', 'Survived']
df_train = df
df = df_train.drop(['Name','SibSp', 'Parch', 'PassengerId'], axis=1)

df.head(5)

In [ ]:
import os
script_folder = os.path.join(os.getcwd(), "prepped_data")
print(script_folder)
os.makedirs(script_folder, exist_ok=True)
df.to_csv('./prepped_data/titanic_prepped.csv', index = False)

## Working with Data

Data can reside in:

    - Local Machine: URI_FILE, URI_FOLDER, MLTABLE, TRITON_MODEL, CUSTOM_MODEL
    - Web
    - Data Storage Services (Bob, ADSL, SQL)
        - https://<account_name>.blob.core.windows.net/<container_name>/path
        - abfss://<file_system>@<account_name>.dfs.core.windows.net/<path>
        - azureml://datastores/<data_store_name>/paths/<path>

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

You can register the a csv file directly from the data directory, we are using a csv file in the directory, but you can leverage data from supported cloud storage using `https`, `abfss` and `wasbs` 

### Register a uri_file

In [ ]:
try:
    registered_data_asset = ml_client.data.get(name='titanic_prepped', version=1)
    print('data asset is registered')
except:
    print('register data asset')
    my_data = Data(
        path="./prepped_data/titanic_prepped.csv",
        type=AssetTypes.URI_FILE,
        description="Titanic CSV",
        name="titanic_prepped",
        version="1",
    )

    ml_client.data.create_or_update(my_data)

### Working with an MLTable

`MLTable` are great for:
- when your data is complex
- you only need a subset of the data
- you will leverage dataset with **AutoML** job which requires tabular data

In [ ]:
import os
script_folder = os.path.join(os.getcwd(), "titanic_prepped_mltable")
print(script_folder)
os.makedirs(script_folder, exist_ok=True)

## Create MLTable Definition file
Note we can exclude columns right in the definition based on the MLTable.schema.json definition

In [ ]:
%%writefile titanic_prepped_mltable/MLTable
$schema: https://azuremlschemas.azureedge.net/latest/MLTable.schema.json 

type: mltable
paths:
    - pattern: ./*.csv

transformations:
  - read_delimited:
      delimiter: ","
      header: all_files_same_headers
      encoding: utf8
  - drop_columns: ["Id"]

In [ ]:
import shutil
data_file = './prepped_data/titanic_prepped.csv'
target   = script_folder + '/titanic_prepped.csv'
shutil.copyfile(data_file, target)

### Loading an MLTable before registration

You should located the `MLTable` file with the data.  You can load an `MLTable` using the `mltable` library

In [ ]:
import mltable

# Note: the uri below can be a local folder or folder located in cloud storage. The folder must contain a valid MLTable file.
script_folder = os.path.join(os.getcwd(), "titanic_prepped_mltable")
print(script_folder)
tbl = mltable.load(uri=script_folder)
tbl.to_pandas_dataframe()

## Register an MLTable

an MLTable can be leveraged as an input to a job or pipeline.  
After it is registered, you can also retrieve it by name.

In [ ]:
try:
    registered_data_asset = ml_client.data.get(name='titanic_prepped_mltable_x2', version=1)
    print('retrieved registered data asset')
except:
    print('registering ml table')
    titanic_data = Data(
        name="titanic_prepped_mltable_x2",
        path='./titanic_prepped_mltable/',
        type=AssetTypes.MLTABLE,
        description="Dataset for titanic",
        tags={"source_type": "file", "source": "ML Engineering"},
        version="1",
    )
    titanic_data = ml_client.data.create_or_update(titanic_data)
    print(f"Dataset with name {titanic_data.name} was registered to workspace, the dataset version is {titanic_data.version}")

In [ ]:
registered_v1_data_asset = ml_client.data.get(name='titanic_prepped_mltable_x2', version='1')
print(registered_v1_data_asset.path)

tbl = mltable.load(uri=registered_v1_data_asset.path)
tbl.to_pandas_dataframe()

## Create Compute 

In [ ]:
from azure.ai.ml.entities import AmlCompute

# specify aml compute name.
cpu_compute_target = "cpu-cluster"

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print("Creating a new cpu compute target...")
    compute = AmlCompute(
        name=cpu_compute_target, size="STANDARD_D2_V2", min_instances=0, max_instances=4, idle_time_before_scale_down = 1800
    )
    ml_client.compute.begin_create_or_update(compute)

## Creating code to generate Basic Model

We will first create a model using the job command, and then leverage the `jobsweep` command with specified parameters for hyperparameter tuning

In [ ]:
script_folder = os.path.join(os.getcwd(), "src")
print(script_folder)
os.makedirs(script_folder, exist_ok=True)

## Create main.py file for running in your command

In [ ]:
%%writefile ./src/main.py
import os
import argparse
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from mlflow.utils.environment import _mlflow_conda_env
from mlflow.tracking import MlflowClient
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score


# define functions
def main(args):
    current_run = mlflow.start_run()
    mlflow.sklearn.autolog(log_models=False)

    # read in data
    df = pd.read_csv(args.titanic_csv)
    model = model_train('Survived', df, args.randomstate)
    mlflow.end_run()

def model_train(LABEL, df, randomstate):
    print('df.columns = ')
    print(df.columns)
    
    df['Embarked'] = df['Embarked'].astype(object)
    df['Loc'] = df['Loc'].astype(object)
    df['Loc'] = df['Sex'].astype(object)
    df['Pclass'] = df['Pclass'].astype(float)
    df['Age'] = df['Age'].astype(float)
    df['Fare'] = df['Fare'].astype(float)
    df['GroupSize'] = df['GroupSize'].astype(float)

    y_raw           = df[LABEL]
    columns_to_keep = ['Embarked', 'Loc', 'Sex','Pclass', 'Age', 'Fare', 'GroupSize']
    X_raw           = df[columns_to_keep]

    print(X_raw.columns)
     # Train test split
    X_train, X_test, y_train, y_test = train_test_split(X_raw, y_raw, test_size=0.2, random_state=randomstate)
    
    #use Logistic Regression estimator from scikit learn
    lg = LogisticRegression(penalty='l2', C=1.0, solver='liblinear')
    preprocessor = buildpreprocessorpipeline(X_train)
    
    #estimator instance
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', lg)], verbose=True)

    model = clf.fit(X_train, y_train)
    
    print('type of X_test = ' + str(type(X_test)))
          
    y_pred = model.predict(X_test)
    
    print('*****X_test************')
    print(X_test)
    
    #get the active run.
    run = mlflow.active_run()
    print("Active run_id: {}".format(run.info.run_id))

    acc = model.score(X_test, y_test )
    print('Accuracy:', acc)
    MlflowClient().log_metric(run.info.run_id, "test_acc", acc)
    
    y_scores = model.predict_proba(X_test)
    auc = roc_auc_score(y_test,y_scores[:,1])
    print('AUC: ' , auc)
    MlflowClient().log_metric(run.info.run_id, "test_auc", auc)
    
    
    # Signature
    signature = infer_signature(X_test, y_test)

    # Conda environment
    custom_env =_mlflow_conda_env(
        additional_conda_deps=["scikit-learn==1.1.3"],
        additional_pip_deps=["mlflow<=1.30.0"],
        additional_conda_channels=None,
    )

    # Sample
    input_example = X_train.sample(n=1)

    # Log the model manually
    mlflow.sklearn.log_model(model, 
                             artifact_path="model", 
                             conda_env=custom_env,
                             signature=signature,
                             input_example=input_example)


    
    return model



def buildpreprocessorpipeline(X_raw):

    categorical_features = X_raw.select_dtypes(include=['object', 'bool']).columns
    numeric_features = X_raw.select_dtypes(include=['float','int64']).columns

    #categorical_features = ['Sex', 'Embarked', 'Loc']
    categorical_transformer = Pipeline(steps=[('onehotencoder', 
                                               OneHotEncoder(categories='auto', sparse=False, handle_unknown='ignore'))])


    #numeric_features = ['Pclass', 'Age', 'Fare', 'GroupSize']    
    numeric_transformer1 = Pipeline(steps=[('scaler1', SimpleImputer(missing_values=np.nan, strategy = 'mean'))])
    

    preprocessor = ColumnTransformer(
        transformers=[
            ('numeric1', numeric_transformer1, numeric_features),
            ('categorical', categorical_transformer, categorical_features)], remainder='drop')
    
    return preprocessor



def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--titanic-csv", type=str)
    parser.add_argument("--randomstate", type=int, default=42)

    # parse args
    args = parser.parse_args()
    print(args)
    # return args
    return args


# run script
if __name__ == "__main__":
    # parse args
    args = parse_args()

    # run main function
    main(args)

## Configure Command

- `display_name` display name for the job
- `description`  the description of the experiment
- `code` path where the code is located
- `command` command to run
- `inputs`  dictionary of name value pairs using `${{inputs.<input_name>}}`
    
    - To use files or folder - using the `Input` class
        
        - `type` defaults to a `uri_folder` but this can be set to `uri_file` or `uri_folder`
        - `path` is the path to the file or folder.  These can be local or remote leveraging **https, http, wasb`
        
            - To use an Azure ML dataset, this would be an Input `Input(type='uri_folder', path='my_dataset:1')`
            
            - `mode` is how the data should be delivered to the compute which include `ro_mount`(default), `rw_mount` and `download`

- `environment`: environment to be used by compute when running command
- `compute`: can be `local`, or a specificed compute name
- `distribution`: distribution to leverage for distributed training scenerios including:
        
    - `Pytorch`
    - `TensorFlow`
    - `MPI`
            
            
          

In [ ]:
# create the command
from azure.ai.ml import command
from azure.ai.ml import Input

my_job = command(
    code="./src",  # local path where the code is stored
    command="python main.py --titanic ${{inputs.titanic}} --randomstate ${{inputs.randomstate}}",
    inputs={
        "titanic": Input(
            type="uri_file",
            path="azureml:titanic_prepped:1",
        ),
        "randomstate": 0,
    },
    environment="job_base_env@latest",
    compute="cpu-cluster",
    display_name="sklearn-titanic",
    # description,
    # experiment_name
)

In [ ]:
script_folder = os.path.join(os.getcwd(), "job")
print(script_folder)
os.makedirs(script_folder, exist_ok=True)

## Run Command with SDK

In [ ]:
# submit the command
returned_job = ml_client.create_or_update(my_job)

In [ ]:
returned_job.services["Studio"].endpoint

In [ ]:
returned_job

In [ ]:
run_id = returned_job.name
print('run_id:' + run_id)
experiment = returned_job.experiment_name
print("experiment:" + experiment)

You can always get the list of runs using ml flow.  Below we will track our process:

In [ ]:
import mlflow
import time

exp = mlflow.get_experiment_by_name(experiment)
last_run = mlflow.search_runs(exp.experiment_id, output_format="list")[-1]

if last_run.info.run_id != run_id:
    print('run ids were not the same - waiting for run id to update')
    time.sleep(5)
    exp = mlflow.get_experiment_by_name(experiment)
    last_run = mlflow.search_runs(exp.experiment_id, output_format="list")[-1]

while last_run.info.status == 'SCHEDULED':
  print('run is being scheduled')
  time.sleep(5)
  last_run = mlflow.search_runs(exp.experiment_id, output_format="list")[-1]

while last_run.info.status == 'RUNNING':
  print('job is being run')
  time.sleep(10)
  last_run = mlflow.search_runs(exp.experiment_id, output_format="list")[-1]

print("run_id:{}".format(last_run.info.run_id))
print('----------')
print("run_id:{}".format(last_run.info.status))

## Create a job.yml file

Note you can also run a command through the CLI.  This is great preperation for MLOps. The file below will allow use to leverage the AML CLI V2 to run this command.  We can and will run it through the CLI, but it is included here for completeness

In [ ]:
%%writefile ./job/job.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandJob.schema.json
code: ../src
command: >-
  python main.py 
  --titanic-csv ${{inputs.titanic}}
  --randomstate ${{inputs.randomstate}}
inputs:
  titanic:
    path: azureml:titanic_prepped:1
    mode: ro_mount
  randomstate: 0   
environment: azureml:job_base_env@latest
compute: azureml:cpu-cluster
experiment_name: titanic-job-example
description: | 
    # Train a classification model on diabetes data using a registered dataset as input.


You can use the CLI to run this command now that you have created a yml file.  Navigate on your compute instace to the folder holding the job.yml file and run the following command in the terminal.

* Note to replace the workspace name in the command with your workspace name (update the value: aml-workspace), and the resource group (update the value: aml-workspace-rg) with your resource group name

```
az login
az ml job create --file job.yml --web --resource-group aml-workspace-rg --workspace-name aml-workspace
```


### List all of the artifacts that were automatically logged as part of your experiment

In [ ]:
import mlflow

print(experiment)
print(run_id)
mlflow.set_experiment(experiment_name=experiment)
client = mlflow.tracking.MlflowClient()
client.list_artifacts(run_id=run_id)

You can download any artifact from the list of artifacts - and display the results

In [ ]:
file_path = client.download_artifacts(
    run_id, path="training_confusion_matrix.png"
)

import matplotlib.pyplot as plt
import matplotlib.image as img

image = img.imread(file_path)
plt.imshow(image)
plt.show()

### Retrieve the Model and consume locally

Given the model was already logged as a job artifact, we can download it locally and run it.  
  



In [ ]:
print(last_run.info.run_id)
pipeline_model = mlflow.sklearn.load_model(f"runs:/{last_run.info.run_id}/model")

In [ ]:
type(pipeline_model)

In [ ]:
script_folder = os.path.join(os.getcwd(), "titanic_prepped_mltable")
print(script_folder)
tbl = mltable.load(uri=script_folder)
df  = tbl.to_pandas_dataframe()
columns_to_keep =  ['Embarked', 'Loc', 'Sex','Pclass', 'Age', 'Fare', 'GroupSize']
X_raw           = df[columns_to_keep]


In [ ]:
results = pipeline_model.predict(X_raw)
print(results)

### Register the Model 

Using the Python SDK V2 - we can register the Model for use.  

Parameters for model registration include:

- `path` - A remote uri or local path pointing at the model
- `name` - A string value
- `description` - A description for the model
- `type` - valid values include: 
    - "custom_model"
    - "mlflow_model" 
    - "triton_model".  
    
* Instead of typing out the `type`, you can use the AssetTypes in the namespace azure.ai.ml.constants as we have done below




In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

run_model = Model(
    path=f"runs:/{last_run.info.run_id}/model",
    name="titanic_model",
    description="Model created from run.",
    type=AssetTypes.MLFLOW_MODEL 
)

ml_client.models.create_or_update(run_model) 

## Hyperparameter Sweep

Configure your experiment to tune your hyperparameters.  The parameters can be discrete or continuous values.

### Sweep Function
The Sweep Function allows you to define:

  - for the job `sampling_algorithm` to be 
    - random
    - grid
    - bayesian
  - `objective`: 
    - primary_metric - the metric must be loggin the the training script using mflow.log_metric()
    - goal - the optimzation goal of the objective.primary_metric
      - maximize
      - minimize
  - `compute` - name of the compute target to excute the job on
  - `limits` - limits for the sweep job

The **Best Child Run** on the Overview screen will show you the best performing child run.

## Update Script to take hyperparameters

We will update the script a bit, and now take in hyperparameters.

```
parser.add_argument("--penalty-term", type=str, default='l1')
parser.add_argument("--C", type=float, default=0.01)
parser.add_argument("--max-iter", type=int, default=100)
```

We have also updated the script to include additional logging so you can gain familiarity with mlflow logging if you are not already familiar.


In [ ]:
import os
script_folder = os.path.join(os.getcwd(), "hyperparametertune")
print(script_folder)
os.makedirs(script_folder, exist_ok=True)

In [ ]:
%%writefile ./hyperparametertune/main.py

import os
import argparse
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature
from mlflow.utils.environment import _mlflow_conda_env
from mlflow.tracking import MlflowClient
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score


# define functions
def main(args):
    # enable auto logging
    current_run = mlflow.start_run()
    mlflow.sklearn.autolog(log_models=False)

    # read in data
    df = pd.read_csv(args.titanic_csv)
    model = model_train('Survived', df, args.penalty_term, args.C, args.max_iter, args.randomstate)
    mlflow.end_run()

def fetch_logged_data(run_id):
    client = mlflow.tracking.MlflowClient()
    data = client.get_run(run_id).data
    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
    artifacts = [f.path for f in client.list_artifacts(run_id, "model")]
    return data.params, data.metrics, tags, artifacts

def model_train(LABEL, df, penalty_term, C, max_iter, randomstate):
    print('df.columns = ')
    print(df.columns)
    
    df['Embarked'] = df['Embarked'].astype(object)
    df['Loc'] = df['Loc'].astype(object)
    df['Loc'] = df['Sex'].astype(object)
    df['Pclass'] = df['Pclass'].astype(float)
    df['Age'] = df['Age'].astype(float)
    df['Fare'] = df['Fare'].astype(float)
    df['GroupSize'] = df['GroupSize'].astype(float)

    y_raw           = df[LABEL]
    columns_to_keep = ['Embarked', 'Loc', 'Sex','Pclass', 'Age', 'Fare', 'GroupSize']
    X_raw           = df[columns_to_keep]
    
    print(X_raw.columns)
     # Train test split

    X_train, X_test, y_train, y_test = train_test_split(X_raw, y_raw, test_size=0.2, shuffle=randomstate)
    
    #use Logistic Regression estimator from scikit learn
    
    lg = LogisticRegression(penalty=penalty_term, C=C, max_iter=max_iter, solver='liblinear')
    preprocessor = buildpreprocessorpipeline(X_train)
    
    #estimator instance
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', lg)], verbose=True)

    model = clf.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    #get the active run.
    run = mlflow.active_run()
    print("Active run_id: {}".format(run.info.run_id))
    

    acc = model.score(X_test, y_test )
    print('Accuracy:', acc)
    MlflowClient().log_metric(run.info.run_id, "test_ACC", acc)
    
    y_scores = model.predict_proba(X_test)
    auc = roc_auc_score(y_test,y_scores[:,1])
    print('AUC: ' , auc)
    MlflowClient().log_metric(run.info.run_id, "test_AUC", auc)
    
    params, metrics, tags, artifacts = fetch_logged_data(run.info.run_id)
    
    print('******************************')
    print(params)
    print('******************************')
    print(metrics)
    print('******************************')
    
    # Signature
    signature = infer_signature(X_test, y_test)

    # Conda environment
    custom_env =_mlflow_conda_env(
        additional_conda_deps=["scikit-learn==1.1.3"],
        additional_pip_deps=["mlflow<=1.30.0"],
        additional_conda_channels=None,
    )

    # Sample
    input_example = X_train.sample(n=1)

    # Log the model manually
    mlflow.sklearn.log_model(model, 
                             artifact_path="model", 
                             conda_env=custom_env,
                             signature=signature,
                             input_example=input_example)


    
    return model


def buildpreprocessorpipeline(X_raw):

    categorical_features = X_raw.select_dtypes(include=['object', 'bool']).columns
    numeric_features = X_raw.select_dtypes(include=['float','int64']).columns

    categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value="missing")),
                                              ('onehotencoder', OneHotEncoder(categories='auto', sparse=False, handle_unknown='ignore'))])
    
    numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('numeric', numeric_transformer, numeric_features),
            ('categorical', categorical_transformer, categorical_features)
        ], remainder="passthrough")
    
    
    return preprocessor



def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--titanic-csv", type=str)
    parser.add_argument("--penalty-term", type=str, default='l1')
    parser.add_argument("--C", type=float, default=0.01)
    parser.add_argument("--max-iter", type=int, default=100)
    parser.add_argument("--randomstate", type=int, default=42)

    # parse args
    args = parser.parse_args()
    print(args)
    # return args
    return args


# run script
if __name__ == "__main__":
    # parse args
    args = parse_args()

    # run main function
    main(args)

## Grid Sampling

In [ ]:
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy, BanditPolicy, TruncationSelectionPolicy

grid_sampling_job_command = command(
    code="./hyperparametertune",  # local path where the code is stored
    command="python main.py --titanic ${{inputs.titanic}} --randomstate ${{inputs.randomstate}}",
    inputs={
        "titanic": Input(
            type="uri_file",
            path="azureml:titanic_prepped:1",
        ),
        "randomstate": 0,
        "penalty_term": 'l1',
        "C": 0.01,
        "max_iter": 100,
    },
    environment="job_base_env@latest",
    compute="cpu-cluster",
    display_name="GridSampling",
)

#Set Parameter expressions
grid_command_job_for_sweep = grid_sampling_job_command(
    penalty_term=Choice(values=['l2', 'l1']),
    C=Choice(values=[0.01, .1, 1.0, 10]),
    max_iter=Choice(values=[10, 100, 150, 200]),
)

In [ ]:
# apply the sweep parameter to obtain the sweep_job
grid_sweep_job = grid_command_job_for_sweep.sweep(
    compute="cpu-cluster",
    sampling_algorithm="grid",
    primary_metric="test_AUC",
    goal="Maximize",
)

#2*4*4 = 32 trial runs, but we will explicity set the max total trials, to see it is not exceeded
grid_sweep_job.set_limits(max_total_trials=60, 
                     max_concurrent_trials=10, 
                     timeout=7200)


In [ ]:
# submit the sweep
returned_sweep_job = ml_client.create_or_update(grid_sweep_job)
# get a URL for the status of the job
returned_sweep_job.services["Studio"].endpoint

In [ ]:
returned_sweep_job

In [ ]:
run_id = returned_sweep_job.name
print('run_id:' + run_id)
experiment = returned_job.experiment_name
print("experiment:" + experiment)

current_experiment=dict(mlflow.get_experiment_by_name(experiment))
experiment_id=current_experiment['experiment_id']
print(experiment_id)

In [ ]:
def get_job_status(experiment_id, run_id):
    df = mlflow.search_runs([experiment_id])
    rslt_df = df[(df['tags.mlflow.parentRunId'] == run_id )]
    rslt_df_finished = rslt_df[rslt_df['status'] == 'FINISHED']

    while rslt_df.shape[0] == 0:
        print('waiting for jobs to register')
        df = mlflow.search_runs([experiment_id])
        rslt_df = df[(df['tags.mlflow.parentRunId'] == run_id )]
        rslt_df_finished = rslt_df[rslt_df['status'] == 'FINISHED']
        time.sleep(5)

    while rslt_df_finished.shape[0] != rslt_df.shape[0]:
        df = mlflow.search_runs([experiment_id])
        rslt_df = df[(df['tags.mlflow.parentRunId'] == run_id )]
        rslt_df_finished = rslt_df[rslt_df['status'] == 'FINISHED']
        status = rslt_df["status"].unique()
        print(status)
        for x in status:
            rslt_df_status = rslt_df[rslt_df['status'] == x]
            print(returned_sweep_job.display_name + ', Number:' + str(x) + " " +  str(rslt_df_status.shape[0]))
        time.sleep(5)

    print('Sweep Job for run:' + run_id + ' Complete')
    
get_job_status(experiment_id, run_id)

## Random Sampling

- Random sampling supports leveraging continuous and discrete hyperparameters.  
- Hyperparameters are selected randomly from the defined search space.  
- Note below we define the following hyperparameters:
    - **penalty_term** to be either 'l1' or 'l2'
    - **C** which is a value from 0.01-10.0
    - **max_iter** is a choice of either 10, 100, 150 or 200

In [ ]:
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy, BanditPolicy, TruncationSelectionPolicy

random_sampling_job_command = command(
    code="./hyperparametertune",  # local path where the code is stored
    command="python main.py --titanic ${{inputs.titanic}} --randomstate ${{inputs.randomstate}}",
    inputs={
        "titanic": Input(
            type="uri_file",
            path="azureml:titanic_prepped:1",
        ),
        "randomstate": 0,
        "penalty_term": 'l1',
        "C": 0.01,
        "max_iter": 100,
    },
    environment="job_base_env@latest",
    compute="cpu-cluster",
    display_name="RandomSampling",
)

# #Set Parameter expressions
# #choice, randint, qlognormal, qnormal, qloguniform, quniform, lognormal, normal, loguniform, uniform
random_command_job_for_sweep = random_sampling_job_command(
    penalty_term=Choice(values=['l2', 'l1']),
    C=Uniform(min_value=0.01, max_value=10.0),
    max_iter=Choice(values=[10, 100, 150, 200]),
)

In [ ]:
# apply the sweep parameter to obtain the sweep_job
random_sweep_job = random_command_job_for_sweep.sweep(
    compute="cpu-cluster",
    sampling_algorithm="random",
    primary_metric="test_AUC",
    goal="Maximize",
)

random_sweep_job.set_limits(max_total_trials=120, 
                     max_concurrent_trials=10, 
                     timeout=7200)

In [ ]:
# submit the sweep
returned_sweep_job = ml_client.create_or_update(random_sweep_job)
# get a URL for the status of the job
returned_sweep_job.services["Studio"].endpoint

In [ ]:
returned_sweep_job

In [ ]:
run_id = returned_sweep_job.name
print('run_id:' + run_id)
experiment = returned_job.experiment_name
print("experiment:" + experiment)

current_experiment=dict(mlflow.get_experiment_by_name(experiment))
experiment_id=current_experiment['experiment_id']
print(experiment_id)

In [ ]:
get_job_status(experiment_id, run_id)

## Random Sampling with Truncation Policy

- As previous we will leverage the random sampling, but now we will apply a truncation policy.
- The truncation selection cancels a percent of the worst performing jobs at each interval based on the selected primary metric.

- Random sampling supports leveraging continuous and discrete hyperparameters.  
- Hyperparameters are selected randomly from the defined search space.  
- Note below we define the following hyperparameters:
    - **penalty_term** to be either 'l1' or 'l2'
    - **C** which is a value from 0.01-10.0
    - **max_iter** is a choice of either 10, 100, 150 or 200

In [ ]:
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy, BanditPolicy, TruncationSelectionPolicy


rnd_sample_trun_command = command(
    code="./hyperparametertune",  # local path where the code is stored
    command="python main.py --titanic ${{inputs.titanic}} --randomstate ${{inputs.randomstate}}",
    inputs={
        "titanic": Input(
            type="uri_file",
            path="azureml:titanic_prepped:1",
        ),
        "randomstate": 0,
        "penalty_term": 'l1',
        "C": 0.01,
        "max_iter": 100,
    },
    environment="job_base_env@latest",
    compute="cpu-cluster",
    display_name="RandomSamplingwTruncationPolicy",
)

# #Set Parameter expressions
# #choice, randint, qlognormal, qnormal, qloguniform, quniform, lognormal, normal, loguniform, uniform
rnd_sample_trun_job_for_sweep = rnd_sample_trun_command(
    penalty_term=Choice(values=['l2', 'l1']),
    C=Uniform(min_value=0.01, max_value=10.0),
    max_iter=Choice(values=[10, 100, 150, 200]),
)

In [ ]:
# apply the sweep parameter to obtain the sweep_job
sweep_job = rnd_sample_trun_job_for_sweep.sweep(
    compute="cpu-cluster",
    sampling_algorithm="random",
    primary_metric="training_roc_auc_score",
    goal="Maximize",
)

sweep_job.set_limits(max_total_trials=120, 
                     max_concurrent_trials=10, 
                     timeout=7200)

#early_termination - Early termination policy to end poorly performing runs. If no termination policy is specified, all configurations are run to completion. 
sweep_job.early_termination = TruncationSelectionPolicy(evaluation_interval= 1, truncation_percentage= 75, delay_evaluation= 1)

In [ ]:
# submit the sweep
returned_sweep_job = ml_client.create_or_update(sweep_job)
# get a URL for the status of the job
returned_sweep_job.services["Studio"].endpoint

In [ ]:
returned_sweep_job

In [ ]:
run_id = returned_sweep_job.name
print('run_id:' + run_id)
experiment = returned_job.experiment_name
print("experiment:" + experiment)

current_experiment=dict(mlflow.get_experiment_by_name(experiment))
experiment_id=current_experiment['experiment_id']
print(experiment_id)

## Getting Status leveraging MLFlow Capabilites

Using MLFlow, we can get back a dataframe that includes all of the trials for a given Sweep Job Run.

In [ ]:
get_job_status(experiment_id, run_id)

## Random Sampling with Median Early Termination Policy

- As previous we will leverage the random sampling, but now we will apply a median early termination policy.
- The median stopping policy is based on the average of the primary metric reported for trials.  
- Random sampling supports leveraging continuous and discrete hyperparameters.  If the primary metric is worse than the median, the job stops.

- Hyperparameters are selected randomly from the defined search space.  
- Note below we define the following hyperparameters:
    - **penalty_term** to be either 'l1' or 'l2'
    - **C** which is a value from 0.01-10.0
    - **max_iter** is a choice of either 10, 100, 150 or 200

In [ ]:
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy, BanditPolicy, TruncationSelectionPolicy


rndsamplemedian_command = command(
    code="./hyperparametertune",  # local path where the code is stored
    command="python main.py --titanic ${{inputs.titanic}} --randomstate ${{inputs.randomstate}}",
    inputs={
        "titanic": Input(
            type="uri_file",
            path="azureml:titanic_prepped:1",
        ),
        "randomstate": 0,
        "penalty_term": 'l1',
        "C": 0.01,
        "max_iter": 100,
    },
    environment="job_base_env@latest",
    compute="cpu-cluster",
    display_name="RandomSamplingwMedianPolicy",
)

# #Set Parameter expressions
# #choice, randint, qlognormal, qnormal, qloguniform, quniform, lognormal, normal, loguniform, uniform
rnd_sample_median_job_for_sweep = rndsamplemedian_command(
    penalty_term=Choice(values=['l2', 'l1']),
    C=Uniform(min_value=0.01, max_value=10.0),
    max_iter=Choice(values=[10, 100, 150, 200]),
)

In [ ]:
# apply the sweep parameter to obtain the sweep_job
sweep_job = rnd_sample_median_job_for_sweep.sweep(
    compute="cpu-cluster",
    sampling_algorithm="random",
    primary_metric="training_roc_auc_score",
    goal="Maximize",
)

sweep_job.set_limits(max_total_trials=120, 
                     max_concurrent_trials=10, 
                     timeout=7200)

#early_termination - Early termination policy to end poorly performing runs. If no termination policy is specified, all configurations are run to completion. 
sweep_job.early_termination = MedianStoppingPolicy(evaluation_interval= 1, delay_evaluation= 2)

In [ ]:
# submit the sweep
returned_sweep_job = ml_client.create_or_update(sweep_job)
# get a URL for the status of the job
returned_sweep_job.services["Studio"].endpoint

In [ ]:
returned_sweep_job

In [ ]:
run_id = returned_sweep_job.name
print('run_id:' + run_id)
experiment = returned_job.experiment_name
print("experiment:" + experiment)

current_experiment=dict(mlflow.get_experiment_by_name(experiment))
experiment_id=current_experiment['experiment_id']
print(experiment_id)

In [ ]:
get_job_status(experiment_id, run_id)

## Random Sampling with Bandit Policy

- As previous we will leverage the random sampling, but now we will apply a bandit early termination policy which will leverage the `slack_factor` to determine if a trial should be ended.
- Random sampling supports leveraging continuous and discrete hyperparameters.  If the primary metric is worse than the median, the job stops.

- Hyperparameters are selected randomly from the defined search space.  
- Note below we define the following hyperparameters:
    - **penalty_term** to be either 'l1' or 'l2'
    - **C** which is a value from 0.01-10.0
    - **max_iter** is a choice of either 10, 100, 150 or 200

In [ ]:
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy, BanditPolicy, TruncationSelectionPolicy


my_job = command(
    code="./hyperparametertune",  # local path where the code is stored
    command="python main.py --titanic ${{inputs.titanic}} --randomstate ${{inputs.randomstate}}",
    inputs={
        "titanic": Input(
            type="uri_file",
            path="azureml:titanic_prepped:1",
        ),
        "randomstate": 0,
        "penalty_term": 'l1',
        "C": 0.01,
        "max_iter": 100,
    },
    environment="job_base_env@latest",
    compute="cpu-cluster",
    display_name="RandomSamplingwBanditPolicy",
)

# #Set Parameter expressions
# #choice, randint, qlognormal, qnormal, qloguniform, quniform, lognormal, normal, loguniform, uniform
command_job_for_sweep = my_job(
    penalty_term=Choice(values=['l2', 'l1']),
    C=Uniform(min_value=0.01, max_value=10.0),
    max_iter=Choice(values=[10, 100, 150, 200]),
)

In [ ]:
# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute="cpu-cluster",
    sampling_algorithm="random",
    primary_metric="training_roc_auc_score",
    goal="Maximize",
)

sweep_job.set_limits(max_total_trials=60, 
                     max_concurrent_trials=10, 
                     timeout=7200)

#early_termination - Early termination policy to end poorly performing runs. If no termination policy is specified, all configurations are run to completion. 
sweep_job.early_termination = BanditPolicy(slack_factor = 0.1, delay_evaluation = 5, evaluation_interval = 1)


In [ ]:
# submit the sweep
returned_sweep_job = ml_client.create_or_update(sweep_job)
# get a URL for the status of the job
returned_sweep_job.services["Studio"].endpoint

In [ ]:
returned_sweep_job

In [ ]:
run_id = returned_sweep_job.name
print('run_id:' + run_id)
experiment = returned_job.experiment_name
print("experiment:" + experiment)

current_experiment=dict(mlflow.get_experiment_by_name(experiment))
experiment_id=current_experiment['experiment_id']
print(experiment_id)

#get status
get_job_status(experiment_id, run_id)

## Bayesian Sampling

- Bayesian sampling leverages how a previous trial did in regard to the primary meteric to determine what to pick for hyperparameter.
- It is recommended to havea  max number of jobs >= 20x  the number of hyperparameters being tuned.

- Baysesian sampling supports: `choice`, `quniform` and `uniform` hyperparameters

In [ ]:
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy, BanditPolicy, TruncationSelectionPolicy


my_job = command(
    code="./hyperparametertune",  # local path where the code is stored
    command="python main.py --titanic ${{inputs.titanic}} --randomstate ${{inputs.randomstate}}",
    inputs={
        "titanic": Input(
            type="uri_file",
            path="azureml:titanic_prepped:1",
        ),
        "randomstate": 0,
        "penalty_term": 'l1',
        "C": 0.01,
        "max_iter": 100,
    },
    environment="job_base_env@latest",
    compute="cpu-cluster",
    display_name="Bayesian",
)

# #Set Parameter expressions
command_job_for_sweep = my_job(
    penalty_term=Choice(values=['l2', 'l1']),
    C=Uniform(min_value=0.01, max_value=10.0),
    max_iter=Choice(values=[10, 100, 150, 200]),
)

In [ ]:
# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute="cpu-cluster",
    sampling_algorithm="bayesian",
    primary_metric="test_AUC",
    goal="Maximize",
)

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=60, max_concurrent_trials=10, timeout=7200)

In [ ]:
# submit the sweep
returned_sweep_job = ml_client.create_or_update(sweep_job)
# get a URL for the status of the job
returned_sweep_job.services["Studio"].endpoint

In [ ]:
returned_sweep_job

In [ ]:
run_id = returned_sweep_job.name
print('run_id:' + run_id)
experiment = returned_job.experiment_name
print("experiment:" + experiment)

current_experiment=dict(mlflow.get_experiment_by_name(experiment))
experiment_id=current_experiment['experiment_id']
print(experiment_id)

#get status
get_job_status(experiment_id, run_id)

In [ ]:
def get_job_run_results(experiment_id, run_id):
    df = mlflow.search_runs([experiment_id])
    rslt_df = df[(df['tags.mlflow.parentRunId'] == run_id )]
    rslt_df_finished = rslt_df[rslt_df['status'] == 'FINISHED']

    while rslt_df.shape[0] == 0:
        print('waiting for jobs to register')
        df = mlflow.search_runs([experiment_id])
        rslt_df = df[(df['tags.mlflow.parentRunId'] == run_id )]
        rslt_df_finished = rslt_df[rslt_df['status'] == 'FINISHED']
        time.sleep(5)

    while rslt_df_finished.shape[0] != rslt_df.shape[0]:
        df = mlflow.search_runs([experiment_id])
        rslt_df = df[(df['tags.mlflow.parentRunId'] == run_id )]
        rslt_df_finished = rslt_df[rslt_df['status'] == 'FINISHED']
        status = rslt_df["status"].unique()
        print(status)
        for x in status:
            rslt_df_status = rslt_df[rslt_df['status'] == x]
            print(returned_sweep_job.display_name + ', Number:' + str(x) + " " +  str(rslt_df_status.shape[0]))
        time.sleep(5)

    rslt_df_status = rslt_df[rslt_df['status'] == 'FINISHED']
    return rslt_df_status

df = get_job_run_results(experiment_id, run_id) 
df

In [ ]:
df = df.sort_values(by='metrics.test_AUC', ascending=False)

In [ ]:
best_run_id = df.iat[0, 0]

In [ ]:
pipeline_model = mlflow.sklearn.load_model(f"runs:/{best_run_id}/model")

In [ ]:
script_folder = os.path.join(os.getcwd(), "titanic_prepped_mltable")
print(script_folder)
tbl = mltable.load(uri=script_folder)
df  = tbl.to_pandas_dataframe()
columns_to_keep =  ['Embarked', 'Loc', 'Sex','Pclass', 'Age', 'Fare', 'GroupSize']
X_raw           = df[columns_to_keep]


In [ ]:
results = pipeline_model.predict(X_raw)
print(results)